# Environment Prep

This notebook sits between:
- tf20_vision_samples/tf2x Obect Detection tutorial
- tensorflow/models/research/object detection  -- which you cannot update

## NOTES - troubleshooting
- the tutorial is written for Windows (ewww) - and, that means '\' in the paths - you need to change them

In [4]:
import os
import sys

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
import numpy as np

In [3]:
print ("tensorflow:", tf.__version__)
# print ("keras:", tf.keras.__version)

tensorflow: 2.4.0


### Globals

In [1]:
# TODO - put logic in here to computer
HOME = os.path.expanduser("~")
PROJECTS = os.path.join(HOME, "projects")
MODELS = os.path.join(HOME, "projects/tensorflow/models")
MODEL_RESEARCH = os.path.join(MODELS, "research")
OBJ_DETECT = os.path.join(MODELS, "research/object_detection")


TRAINING_DEMO = os.path.join(PROJECTS, "tensorflow/workspace/training_demo")
PIPELINE_CONFIG = os.path.join(TRAINING_DEMO, "models/my_ssd_resnet50_v1_fpn","laptop_pipeline.config")
TRAINED_EXPORT = os.path.join(TRAINING_DEMO, "exported-models/")
INFERENCE_GRAPH = os.path.join(TRAINING_DEMO, "inference_graph")

NameError: name 'os' is not defined

In [5]:
!ls {TRAINING_DEMO}/inference_graph

## DATA

you only need to move the data once - it will be there between restarts

### tfrecords
the tutorial says you need 2 new files under the training_demo/annotations folder, named 
- test.record
- train.record

In [ ]:
import os

In [ ]:
!pwd

In [ ]:
!ls /home/ec2-user/SageMaker/tensorflow/workspace/training_demo/annotations

## Model
download the ResNet50 640x640 model  

copy the pipeline.config file to my* folder.   Then edit locally with gedit.

In [ ]:
# you should see checkpoint, pipeline.config, saved_model
!ls /home/ec2-user/SageMaker/tensorflow/workspace/training_demo/pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/

In [ ]:
!ls /home/ec2-user/SageMaker/tensorflow/workspace/training_demo/models/my_ssd_resnet50_v1_fpn -l

## TF 2.x GPU
Common Problems
- if list_physical_devices shows GPU and gpu_device_name is none, then you probably have a CUDA version issue - check CUDA version, PATH & LD_LIBRARY path in ./bashrc.     e.g.   path needs to include CUDA 10.1 for TF2.2
- check your version of TensorFlow, one of the steps automatically upgrades from TF 2.x to 2.4.   Thus, you need CUDA 11.0 installed.   However, I didn't notice any path requirements...

In [ ]:
import tensorflow as tf

In [ ]:
print (tf.__version__)

In [ ]:
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

In [ ]:
physical_devices= tf.config.experimental.list_physical_devices('GPU')
print(len(physical_devices))

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
print(tf.reduce_sum(tf.random.normal([1000, 1000])))

## Tutorial - Object Detection API Setup

In [ ]:
!pip install pycocotools

In [ ]:
import pycocotools

### API

commom problems:

#### ASUS


#### Dell Inspiron

```ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
paramiko 2.4.2 requires cryptography>=1.5, which is not installed.
fabric 2.4.0 requires cryptography>=1.1, which is not installed.
aws 0.2.5 requires boto, which is not installed.
tensorflow-gpu 2.2.0 requires tensorboard<2.3.0,>=2.2.0, but you have tensorboard 2.4.0 which is incompatible.
tensorflow-gpu 2.2.0 requires tensorflow-estimator<2.3.0,>=2.2.0, but you have tensorflow-estimator 2.4.0 which is incompatible.

```
- pip install crytography  # 3.3.1
- pip install boto # 2.49.0
- DON'T mess with tensorflow* dependencies - because this script is in some kind of loop!   I had 2.2 then it uninstalled and upgraded to 2.4 so the fact that it is complaining about 2.4 is kind of bogus.   I suppose you could back it down and it would probably work.  BUT - I did nothing and then reran the script and it worked fine

Successfully installed object-detection-0.1

In [ ]:
original_cwd = cwd = os.getcwd()
print ("Original working directory:", original_cwd)

In [ ]:
os.chdir('/home/ec2-user/SageMaker/tensorflow/models/research/')
cwd = os.getcwd() 
print("Current working directory is:", cwd)

In [ ]:
# make sure your kernel is correct:  conda_tensorflow2_p36
!python -m pip install .

### Test the API
you should be in models/research

Possible Problems:
- GPU error - memory;   you may need to close this notebook and others to free up GPU memory

In [ ]:
!python object_detection/builders/model_builder_tf2_test.py

In [ ]:
os.chdir(original_cwd)
cwd = os.getcwd() 
print("back to original working directory:", cwd)

## train

speed benchmarks
- SageMaker - 27.7 minutes!! per step - that was CPU only becuase CUDA 11.0 is not installed
- ASUS 1080 TI - 0.7 sec (8 images/batch)
- Dell Inspiron 1060 - 1.0  (4 images/batch, memory warnings)

In [ ]:
!cp /home/ec2-user/SageMaker/tensorflow/models/research/object_detection/model_main_tf2.py .

In [ ]:
os.chdir('/home/ec2-user/SageMaker/tensorflow/workspace/training_demo/')
cwd = os.getcwd() 
print("Current working directory is:", cwd)

In [ ]:
!python model_main_tf2.py --model_dir=models/my_ssd_resnet50_v1_fpn --pipeline_config_path=models/my_ssd_resnet50_v1_fpn/pipeline.config

## export

https://medium.com/@sebastingarcaacosta/how-to-export-a-tensorflow-2-x-keras-model-to-a-frozen-and-optimized-graph-39740846d9eb

from checkpoint -> saved model

In [5]:
!ls {HOME}/projects/tensorflow/models/research/object_detection/exp*.py

/home/jay/projects/tensorflow/models/research/object_detection/exporter_lib_tf2_test.py
/home/jay/projects/tensorflow/models/research/object_detection/exporter_lib_v2.py
/home/jay/projects/tensorflow/models/research/object_detection/exporter_main_v2.py
/home/jay/projects/tensorflow/models/research/object_detection/exporter.py
/home/jay/projects/tensorflow/models/research/object_detection/exporter_tf1_test.py
/home/jay/projects/tensorflow/models/research/object_detection/export_inference_graph.py
/home/jay/projects/tensorflow/models/research/object_detection/export_tflite_graph_lib_tf2.py
/home/jay/projects/tensorflow/models/research/object_detection/export_tflite_graph_lib_tf2_test.py
/home/jay/projects/tensorflow/models/research/object_detection/export_tflite_graph_tf2.py
/home/jay/projects/tensorflow/models/research/object_detection/export_tflite_ssd_graph_lib.py
/home/jay/projects/tensorflow/models/research/object_detection/export_tflite_ssd_graph_lib_tf1_test.py
/home/jay/projects/

In [6]:
os.chdir(TRAINING_DEMO)
cwd = os.getcwd() 
print("Current working directory is:", cwd)

Current working directory is: /home/jay/projects/tensorflow/workspace/training_demo


### NOTE - multiple model exports here
- original (to test with)
- your resnet50

In [5]:
PIPELINE_CONFIG_PATH = os.path.join(TRAINING_DEMO, "pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/pipeline.config/")
TRAINED_CHECKPOINT_DIR = os.path.join(TRAINING_DEMO, "pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/")
OUTPUT_DIR = os.path.join(TRAINING_DEMO, "exported-models/orig_resnet50_640/" )

!ls {PIPELINE_CONFIG_PATH}
!ls {TRAINED_CHECKPOINT_DIR}
!ls {OUTPUT_DIR}

ls: cannot access '/home/jay/projects/tensorflow/workspace/training_demo/pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/pipeline.config/': Not a directory
checkpoint  ckpt-0.data-00000-of-00001	ckpt-0.index
checkpoint  inference  pipeline.config	saved_model


In [11]:
!python ./exporter_main_v2.py  \
--input_type image_tensor \
--pipeline_config_path {PIPELINE_CONFIG_PATH} \
--trained_checkpoint_dir {TRAINED_CHECKPOINT_DIR} \
--output_directory {OUTPUT_DIR}

2020-12-26 11:16:35.493338: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2020-12-26 11:16:37.100070: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2020-12-26 11:16:37.120003: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2020-12-26 11:16:37.167266: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-12-26 11:16:37.167711: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1060 with Max-Q Design computeCapability: 6.1
coreClock: 1.3415GHz coreCount: 10 deviceMemorySize: 5.94GiB deviceMemoryBandwidth: 178.99GiB/s
2020-12-26 11:16:37.167735: I tensorflow/stream_executor

INFO:tensorflow:Unsupported signature for serialization: (([(<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fb135b86610>, TensorSpec(shape=(None, 80, 80, 512), dtype=tf.float32, name='image_features/0/1')), (<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fb135b86c10>, TensorSpec(shape=(None, 40, 40, 1024), dtype=tf.float32, name='image_features/1/1')), (<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fb135b86dd0>, TensorSpec(shape=(None, 20, 20, 2048), dtype=tf.float32, name='image_features/2/1'))], True), {}).
I1226 11:17:28.522341 140403954206528 def_function.py:1170] Unsupported signature for serialization: (([(<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fb135b86610>, TensorSpec(shape=(None, 80, 80, 512), dtype=tf.float32, name='image_features/0/1')), (<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fb135b86c10>, TensorSpec(shape=(None, 40, 40, 1024), dtype=tf.float32, name

INFO:tensorflow:Unsupported signature for serialization: (([(<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fb135b86610>, TensorSpec(shape=(None, 80, 80, 512), dtype=tf.float32, name='image_features/0/1')), (<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fb135b86c10>, TensorSpec(shape=(None, 40, 40, 1024), dtype=tf.float32, name='image_features/1/1')), (<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fb135b86dd0>, TensorSpec(shape=(None, 20, 20, 2048), dtype=tf.float32, name='image_features/2/1'))], True), {}).
I1226 11:17:32.637524 140403954206528 def_function.py:1170] Unsupported signature for serialization: (([(<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fb135b86610>, TensorSpec(shape=(None, 80, 80, 512), dtype=tf.float32, name='image_features/0/1')), (<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7fb135b86c10>, TensorSpec(shape=(None, 40, 40, 1024), dtype=tf.float32, name

INFO:tensorflow:Assets written to: /home/jay/projects/tensorflow/workspace/training_demo/exported-models/orig_resnet50_640/saved_model/assets
I1226 11:17:38.944477 140403954206528 builder_impl.py:775] Assets written to: /home/jay/projects/tensorflow/workspace/training_demo/exported-models/orig_resnet50_640/saved_model/assets
INFO:tensorflow:Writing pipeline config file to /home/jay/projects/tensorflow/workspace/training_demo/exported-models/orig_resnet50_640/pipeline.config
I1226 11:17:39.796017 140403954206528 config_util.py:254] Writing pipeline config file to /home/jay/projects/tensorflow/workspace/training_demo/exported-models/orig_resnet50_640/pipeline.config


In [12]:
!ls {OUTPUT_DIR}

ls: cannot access '{OUTPUT_DIR}': No such file or directory


### export MY resnet50_640

In [14]:
# my_ssd_resnet50_v1_fpn
# LAPTOP config
!ls
!python ./exporter_main_v2.py \
--input_type image_tensor \
--pipeline_config_path ./models/my_ssd_resnet50_v1_fpn/laptop_pipeline.config \
--trained_checkpoint_dir ./models/my_ssd_resnet50_v1_fpn/ \
--output_directory ./exported-models/resnet50_640

'$'		       export_inference_graph.py   models
 annotations	       images			   pre-trained-models
 exported-models       inference_graph		   README.md
 exporter_main_v2.py   model_main_tf2.py
2020-12-26 11:22:43.005551: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2020-12-26 11:22:44.579273: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2020-12-26 11:22:44.580115: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2020-12-26 11:22:44.608589: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-12-26 11:22:44.609129: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1060 w

INFO:tensorflow:Unsupported signature for serialization: (([(<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7ff069a337d0>, TensorSpec(shape=(None, 80, 80, 512), dtype=tf.float32, name='image_features/0/1')), (<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7ff069a33d90>, TensorSpec(shape=(None, 40, 40, 1024), dtype=tf.float32, name='image_features/1/1')), (<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7ff069a33fd0>, TensorSpec(shape=(None, 20, 20, 2048), dtype=tf.float32, name='image_features/2/1'))], False), {}).
I1226 11:23:26.057494 140681203464000 def_function.py:1170] Unsupported signature for serialization: (([(<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7ff069a337d0>, TensorSpec(shape=(None, 80, 80, 512), dtype=tf.float32, name='image_features/0/1')), (<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7ff069a33d90>, TensorSpec(shape=(None, 40, 40, 1024), dtype=tf.float32, nam

INFO:tensorflow:Unsupported signature for serialization: (([(<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7ff069a337d0>, TensorSpec(shape=(None, 80, 80, 512), dtype=tf.float32, name='image_features/0/1')), (<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7ff069a33d90>, TensorSpec(shape=(None, 40, 40, 1024), dtype=tf.float32, name='image_features/1/1')), (<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7ff069a33fd0>, TensorSpec(shape=(None, 20, 20, 2048), dtype=tf.float32, name='image_features/2/1'))], False), {}).
I1226 11:23:29.988829 140681203464000 def_function.py:1170] Unsupported signature for serialization: (([(<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7ff069a337d0>, TensorSpec(shape=(None, 80, 80, 512), dtype=tf.float32, name='image_features/0/1')), (<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7ff069a33d90>, TensorSpec(shape=(None, 40, 40, 1024), dtype=tf.float32, nam

INFO:tensorflow:Assets written to: ./exported-models/resnet50_640/saved_model/assets
I1226 11:23:37.534722 140681203464000 builder_impl.py:775] Assets written to: ./exported-models/resnet50_640/saved_model/assets
INFO:tensorflow:Writing pipeline config file to ./exported-models/resnet50_640/pipeline.config
I1226 11:23:38.282922 140681203464000 config_util.py:254] Writing pipeline config file to ./exported-models/resnet50_640/pipeline.config


## Export to Inference Graph (frozen_graph.pb)

In [6]:
INFERENCE_DIR = os.path.join(TRAINING_DEMO, "exported-models/orig_resnet50_640/inference")

In [7]:
!python {MODEL_RESEARCH}/object_detection/export_inference_graph.py \
  --input_type image_tensor \
  --pipeline_config_path="{PIPELINE_CONFIG}" \ \
  --trained_checkpoint_prefix="{TRAINED_CHECKPOINT_DIR}/ckpt-0" \
  --output_directory="{INFERENCE_DIR}"

2020-12-26 11:38:41.071029: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2020-12-26 11:38:42.609242: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2020-12-26 11:38:42.609895: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2020-12-26 11:38:42.639282: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-12-26 11:38:42.639624: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1060 with Max-Q Design computeCapability: 6.1
coreClock: 1.3415GHz coreCount: 10 deviceMemorySize: 5.94GiB deviceMemoryBandwidth: 178.99GiB/s
2020-12-26 11:38:42.639665: I tensorflow/stream_executor

In [ ]:
!ls {TRAINING_DEMO}/exported-models/resnet50_640
!ls {TRAINING_DEMO}/exported-models/resnet50_640/checkpoint -l
!ls {TRAINING_DEMO}/exported-models/resnet50_640/saved_model -l

In [ ]:
# no $ in the command line
# add $ in the passed variables?

!python {MODEL_RESEARCH}/object_detection/export_inference_graph.py \
  --input_type image_tensor \
  --pipeline_config_path="{PIPELINE_CONFIG}" \ \
  --trained_checkpoint_prefix="{TRAINED_EXPORT}/ckpt-0" \
  --output_directory="{INFERENCE_GRAPH}"

In [ ]:
#path of the directory where you want to save your model
frozen_out_path = INFERENCE_GRAPH
# name of the .pb file
frozen_graph_filename = "frozen_graph"
model_path = '/home/jay/projects/tensorflow/workspace/training_demo/exported-models/resnet50_640/saved_model'

loaded_model = tf.keras.models.load_model('/home/jay/projects/tensorflow/workspace/training_demo/exported-models/resnet50_640/saved_model/')

# tf.keras.models.load_model(model_path, custom_objects=None, compile=True, options=None)

## Keras - load model

https://www.tensorflow.org/api_docs/python/tf/keras/models/load_model

https://github.com/tensorflow/tensorflow/issues/43527


In [6]:
# original resnet50_640
SAVED_MODEL_PATH = os.path.join(TRAINING_DEMO, "pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model")
print (type(SAVED_MODEL_PATH))
! ls {SAVED_MODEL_PATH}

<class 'str'>
assets	saved_model.pb	variables


In [18]:
# keras model class
model = tf.keras.models.load_model("/home/jay/projects/tensorflow/workspace/training_demo/pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model")

IndexError: list index out of range

In [8]:
model = tf.saved_model.load(export_dir="/home/jay/projects/tensorflow/workspace/training_demo/pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model", tags=None, options=None)

In [15]:
# <class 'tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject'>
model = tf.saved_model.load(SAVED_MODEL_PATH)
print (type(model))

<class 'tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject'>


In [16]:
model = tf.keras.models.load_model(
    {SAVED_MODEL_PATH}, custom_objects=None, compile=True
)

TypeError: expected str, bytes or os.PathLike object, not set

In [17]:
# Convert Keras model to ConcreteFunction
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))

AttributeError: '_UserObject' object has no attribute 'inputs'

In [ ]:
!ls {model_path} -l
model = tf.compat.v2.saved_model.load(export_dir=model_path)
print (type(model))

In [10]:
# Convert Keras model to ConcreteFunction
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))

AttributeError: '_UserObject' object has no attribute 'inputs'

In [14]:
!saved_model_cli show --dir {SAVED_MODEL_PATH} --all

2020-12-26 13:24:35.605187: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0

MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_tensor'] tensor_info:
        dtype: DT_UINT8
        shape: (1, -1, -1, 3)
        name: serving_default_input_tensor:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['detection_anchor_indices'] tensor_info:
        dtype: DT_FLOAT
        shape: (1, 100)
        name: StatefulPartitionedCall:0
    

## Export TF 1.15

In [19]:
# from training demo 
# cut/past

PIPELINE_CONFIG_PATH = os.path.join(TRAINING_DEMO, "pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/pipeline.config/")
TRAINED_CHECKPOINT_PATH = os.path.join(TRAINING_DEMO, "pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0")
OUTPUT_DIR = os.path.join(TRAINING_DEMO, "exported-models/orig_resnet50_640/inference" )

In [ ]:
add object_detection to your python path

In [ ]:
python export_inference_graph.py \
    --input_type image_tensor \
    --pipeline_config_path path/to/ssd_inception_v2.config \
    --trained_checkpoint_prefix path/to/model.ckpt \
    --output_directory path/to/exported_model_directory

In [3]:
! python /home/jay/projects/tensorflow/workspace/training_demo/export_inference_graph.py \
    --input_type image_tensor \
    --pipeline_config_path /home/jay/projects/tensorflow/workspace/training_demo/pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/pipeline.config/ \
    --trained_checkpoint_prefix /home/jay/projects/tensorflow/workspace/training_demo/pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0 \
    --output_directory /home/jay/projects/tensorflow/workspace/training_demo/exported-models/orig_resnet50_640/inference

2020-12-26 13:53:36.237415: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2020-12-26 13:53:37.739246: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2020-12-26 13:53:37.739886: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2020-12-26 13:53:37.770611: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-12-26 13:53:37.771166: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 1060 with Max-Q Design computeCapability: 6.1
coreClock: 1.3415GHz coreCount: 10 deviceMemorySize: 5.94GiB deviceMemoryBandwidth: 178.99GiB/s
2020-12-26 13:53:37.771211: I tensorflow/stream_executor